In [ ]:
pip install --upgrade ultralytics

In [1]:
from ultralytics import YOLO
import torch

print("Using GPU:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0))

model = YOLO('yolo11n-cls.pt')  # or yolov11s.pt / yolov11m.pt for larger variants

model.train(
    data='Road_Damage',
    epochs=50,        # your hard cap
    imgsz=640,
    device='cuda',
)


Using GPU: True
Device: NVIDIA GeForce RTX 4060 Laptop GPU
Ultralytics 8.3.127  Python-3.9.21 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=Road_Damage, degrees=0.0, deterministic=True, device=cuda:0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train19, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_ma

train: Scanning C:\Users\jazzb\ImageDetection-Yolov9\Road_Damage\train... 1526 images, 0 corrupt: 100%|██████████| 1526/1526 [00:00<00:00, 3190.29it/s]

train: New cache created: C:\Users\jazzb\ImageDetection-Yolov9\Road_Damage\train.cache


val: Fast image access  (ping: 0.10.0 ms, read: 238.3102.5 MB/s, size: 212.7 KB)


val: Scanning C:\Users\jazzb\ImageDetection-Yolov9\Road_Damage\val... 67 images, 0 corrupt: 100%|██████████| 67/67 [00:00<00:00, 2136.42it/s]

...

                   all      0.806      0.985



50 epochs completed in 0.114 hours.
Optimizer stripped from runs\classify\train19\weights\last.pt, 3.2MB
Optimizer stripped from runs\classify\train19\weights\best.pt, 3.2MB

Validating runs\classify\train19\weights\best.pt...
Ultralytics 8.3.127  Python-3.9.21 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLO11n-cls summary (fused): 47 layers, 1,533,710 parameters, 0 gradients, 3.2 GFLOPs
train: C:\Users\jazzb\ImageDetection-Yolov9\Road_Damage\train... found 1526 images in 6 classes  
val: C:\Users\jazzb\ImageDetection-Yolov9\Road_Damage\val... found 67 images in 6 classes  
test: C:\Users\jazzb\ImageDetection-Yolov9\Road_Damage\test... found 39 images in 6 classes  


               classes   top1_acc   top5_acc: 100%|██████████| 3/3 [00:00<00:00, 21.42it/s]


                   all      0.836          1
Speed: 0.5ms preprocess, 1.2ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\train19


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000248F3633970>
curves: []
curves_results: []
fitness: 0.9179104566574097
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.8358209133148193, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9179104566574097}
save_dir: WindowsPath('runs/classify/train19')
speed: {'preprocess': 0.49621492537392164, 'inference': 1.1565567164182797, 'loss': 0.0004850746273377944, 'postprocess': 0.001319402985646137}
task: 'classify'
top1: 0.8358209133148193
top5: 1.0

In [ ]:
from ultralytics import YOLO
import torch

print("Using GPU:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0))

model = YOLO('yolo11n.pt')  # or yolov11s.pt / yolov11m.pt for larger variants

model.train(
    data='road_damage.yaml',
    epochs=50,        # your hard cap
    imgsz=640,
    device='cuda',
)


Using GPU: True
Device: NVIDIA GeForce RTX 4060 Laptop GPU
Ultralytics 8.3.127  Python-3.9.21 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=road_damage.yaml, degrees=0.0, deterministic=True, device=cuda:0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train34, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_m

train: Scanning C:\Users\jazzb\ImageDetection-Yolov9\Road_Damage-OBJD\train\labels... 1584 images, 4 backgrounds, 0 corrupt: 100%|██████████| 1584/1584 [00:02<00:00, 565.61it/s]


train: New cache created: C:\Users\jazzb\ImageDetection-Yolov9\Road_Damage-OBJD\train\labels.cache
val: Fast image access  (ping: 0.40.1 ms, read: 13.34.8 MB/s, size: 156.6 KB)


val: Scanning C:\Users\jazzb\ImageDetection-Yolov9\Road_Damage-OBJD\val\labels... 74 images, 0 backgrounds, 0 corrupt: 100%|██████████| 74/74 [00:00<00:00, 270.26it/s]

val: New cache created: C:\Users\jazzb\ImageDetection-Yolov9\Road_Damage-OBJD\val\labels.cache


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      46/50      2.68G      1.298      1.468      1.681         26        640: 100%|██████████| 99/99 [00:13<00:00,  7.23it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  6.92it/s]

                   all         74        111      0.503      0.453      0.402      0.186



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      47/50      2.68G      1.278      1.443      1.669         24        640:  74%|███████▎  | 73/99 [00:09<00:03,  6.62it/s]

In [ ]:
from IPython.display import Image

In [ ]:
Image("runs/detect/train33/results.png", width=600)

In [24]:
results = model("test image/potholes.jpg", save=True, conf=0.5)
results[0].show()


image 1/1 C:\Users\jazzb\ImageDetection-Yolov9\test image\potholes.jpg: 448x640 1 potholes, 72.4ms
Speed: 1.8ms preprocess, 72.4ms inference, 1.4ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs\detect\predict2


In [27]:
results = model.predict(source='test image/crack.jpeg', augment=True, conf=0.25, iou=0.45)
results[0].show()


image 1/1 C:\Users\jazzb\ImageDetection-Yolov9\test image\crack.jpeg: 448x640 (no detections), 103.5ms
Speed: 6.6ms preprocess, 103.5ms inference, 0.7ms postprocess per image at shape (1, 3, 448, 640)
